# Advanced NDArray tutorial: NumPy arrays and NDArrays

In the [NDArray basics tutorial](02.ndarray-basics.html) we saw how to create NDArray and set slices to the values of a NumPy array. But you can also create a NDArray from a NumPy array with the `asarray` function. Let's check this out!

In [1]:
import blosc2
import numpy as np

## Creating a NDArray from a NumPy array

We will first create a NumPy array and create a NDArray from it.

In [2]:
shape = (100, 100, 100)
dtype = np.float64
nparray = np.linspace(0, 100, np.prod(shape), dtype=dtype).reshape(shape)
b2ndarray = blosc2.asarray(nparray)
print(b2ndarray.info)

type    : NDArray
shape   : (100, 100, 100)
chunks  : (100, 100, 100)
blocks  : (32, 32, 32)
dtype   : float64
cratio  : 16.00
cparams : {'blocksize': 262144,
 'clevel': 1,
 'codec': <Codec.ZSTD: 5>,
 'codec_meta': 0,
 'filters': [<Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.NOFILTER: 0>,
             <Filter.SHUFFLE: 1>],
 'filters_meta': [0, 0, 0, 0, 0, 0],
 'nthreads': 2,
 'splitmode': <SplitMode.ALWAYS_SPLIT: 1>,
 'typesize': 8,
 'use_dict': 0}
dparams : {'nthreads': 2}



## Building a NDArray from a buffer

Furthermore, you can create a NDArray filled with data from a buffer:

In [3]:
buffer = bytes(np.random.normal(0, 1, np.prod(shape)) * 8)
b2ndarray = blosc2.frombuffer(buffer, shape, dtype=dtype)
print("Compression ratio:", b2ndarray.schunk.cratio)
b2ndarray[:5, :5, :5]

Compression ratio: 2.34648018517612


array([[[ 2.90142294e-01,  6.00364532e+00,  1.42442172e+00,
         -4.99554555e+00,  6.19454573e-01],
        [ 1.00020988e+01, -1.76757106e+01,  2.51317170e+00,
         -1.90917192e+00, -2.70592236e+00],
        [ 7.37423771e+00,  1.04212462e+01,  8.04882697e+00,
         -1.43199549e+01, -3.45910950e+00],
        [ 1.27661906e+01, -9.43514916e+00,  8.88374464e+00,
          2.69273166e+00, -3.77710724e+00],
        [ 9.14030559e+00,  1.75787001e+00,  9.52035273e+00,
          6.25335241e-01, -6.72120860e+00]],

       [[ 4.75364992e+00, -9.35220828e+00, -7.56732236e+00,
          1.23780719e+01,  5.68336660e+00],
        [ 1.05150495e+00, -4.04485813e+00,  6.78892180e-02,
          1.27232322e+01, -9.56710456e+00],
        [ 7.04345822e+00,  3.19199500e+00, -6.60152568e+00,
         -4.03375561e+00,  1.06913026e+00],
        [ 2.65452758e+00, -5.46139871e+00, -1.48943101e-01,
          7.85667920e+00, -7.58526315e+00],
        [-1.01251426e+00, -5.18814832e+00,  6.13525807e+00,
  

## Serializing NumPy arrays

If what you want is to create a serialized, compressed version of a NumPy array, you can use the newer (and faster) functions to store it either in-memory or on-disk.  The specification of such a contiguous compressed representation, aka **cframe** can be seen [here](https://github.com/Blosc/c-blosc2/blob/main/README_CFRAME_FORMAT.rst).

### In-memory

For obtaining an in-memory representation, you can use `pack_tensor`. In comparison with its former version (`pack_array`), it is way faster and does not have the 2 GB size limitation:

In [4]:
shape = (2**10, ) * 3
np_array = np.arange(np.prod(shape), dtype=np.int32)  # 4 GB array

packed_arr2 = blosc2.pack_tensor(np_array)
unpacked_arr2 = blosc2.unpack_tensor(packed_arr2)

### On-disk

To store the serialized buffer on-disk you want to use `save_tensor` and `load_tensor`:

In [5]:
blosc2.save_tensor(np_array, urlpath="ondisk_array.b2frame", mode="w")
np_array2 = blosc2.load_tensor("ondisk_array.b2frame")
np.array_equal(np_array, np_array2)

True